# Title basics : extraction only for movies and TV  movies

In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime

In [2]:
url = 'https://datasets.imdbws.com/title.basics.tsv.gz'


In [3]:
chunk_size = 1000
iterator = pd.read_csv(url, sep='\t', compression='gzip', iterator=True, chunksize=chunk_size, na_values=["\\N"])

In [4]:
cleaned_chunks = []
for chunk in iterator:

    if 'endYear' in chunk.columns:
        chunk.drop(columns=['endYear'], inplace=True)      #drop endYear
    chunk.drop(columns=['isAdult','originalTitle','runtimeMinutes'], inplace=True) #drop unused columns

    if 'genres' in chunk.columns:
        chunk.dropna(subset=['genres'], inplace=True)        #suppriner les colonnes ou le genre est nulls

    if 'runtimeMinutes' in chunk.columns:
        chunk['runtimeMinutes'] = pd.to_numeric(chunk['runtimeMinutes'], errors='coerce')      #convertir les valeurs de runtimeMinutes en numerique
        chunk.dropna(subset=['runtimeMinutes'], inplace=True)        #supprimer les lignes ou runtimeLinutes est null
        
    cleaned_chunks.append(chunk)

In [5]:
full_clean_basic_title = pd.concat(cleaned_chunks, ignore_index=True)      #concaterner mes chunks pour avoir un full dataset clean
print(full_clean_basic_title.info())
print(full_clean_basic_title.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10678001 entries, 0 to 10678000
Data columns (total 5 columns):
 #   Column        Dtype  
---  ------        -----  
 0   tconst        object 
 1   titleType     object 
 2   primaryTitle  object 
 3   startYear     float64
 4   genres        object 
dtypes: float64(1), object(4)
memory usage: 407.3+ MB
None
          startYear
count  9.345547e+06
mean   2.006254e+03
std    1.988354e+01
min    1.874000e+03
25%    2.002000e+03
50%    2.013000e+03
75%    2.019000e+03
max    2.031000e+03


In [6]:
movie_types = ['movie', 'tvMovie']
df_title_basics_movies = full_clean_basic_title[full_clean_basic_title['titleType'].isin(movie_types)]
display(df_title_basics_movies)

df_title_basics_movies.info()

,tconst,titleType,primaryTitle,startYear,genres
8,tt0000009,movie,Miss Jerry,1894.0,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897.0,"Documentary,News,Sport"
569,tt0000574,movie,The Story of the Kelly Gang,1906.0,"Action,Adventure,Biography"
586,tt0000591,movie,The Prodigal Son,1907.0,Drama
609,tt0000615,movie,Robbery Under Arms,1907.0,Drama
...,...,...,...,...,...
10677920,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,2007.0,Documentary
10677925,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,2015.0,Drama
10677931,tt9916706,movie,Dankyavar Danka,2013.0,Comedy
10677941,tt9916730,movie,6 Gunn,2017.0,Drama


<class 'pandas.core.frame.DataFrame'>
Index: 754983 entries, 8 to 10677951
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   tconst        754983 non-null  object 
 1   titleType     754983 non-null  object 
 2   primaryTitle  754981 non-null  object 
 3   startYear     656434 non-null  float64
 4   genres        754983 non-null  object 
dtypes: float64(1), object(4)
memory usage: 34.6+ MB


In [7]:
df_title_basics_movies['titleType'].unique()

array(['movie', 'tvMovie'], dtype=object)

In [8]:
#no need to keep title type ?
# subset_title_basics_movies.drop(['titleType'], axis=1, inplace=True)
df_title_basics_movies.reset_index(drop=True, inplace=True)
df_title_basics_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754983 entries, 0 to 754982
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   tconst        754983 non-null  object 
 1   titleType     754983 non-null  object 
 2   primaryTitle  754981 non-null  object 
 3   startYear     656434 non-null  float64
 4   genres        754983 non-null  object 
dtypes: float64(1), object(4)
memory usage: 28.8+ MB


In [103]:
# df_title_basics_movies.to_csv('df_title_basics_movies.csv', sep=';', index=False)
# !cp subset_moviessep.csv "drive/My Drive/"

### TMDB:On récupere les données complémentaires tmdb pour le productions_countries

Donwloaded csv from link : https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies
Then extraction of relevant columns


In [9]:
#https://www.nro.net/list-of-country-codes-in-the-lacnic-region/
country_code_list = ['AR','AW','BZ','BO','BR','CL','CO',
                     'CR','CU','DO','EC','SV','FK','GF',
                     'GT','GY','HT','HN','MX','AN','NI',
                     'PA','PY','PE','GS','SR','TT','UY','VE',
                     #'FR'
                     ]
###################################""
#
country_names = ['Argentina','Aruba','Bolivia','Brasil','Chile','Colombia',
                     'Costa Rica','Cuba','Dominican Republic','Ecuador','El Salvador',
                     'Guatemala',
                     'Guyana','Haiti','Honduras','Mexico',
                     'Nicaragua',
                     'Panama','Peru','Uruguay','Venezuela',
                     #'FR'
                     ]

In [10]:

url= "TMDB_movie_dataset_v11.csv"
df_tmdb = pd.read_csv(url,low_memory=False)
df_tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118015 entries, 0 to 1118014
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1118015 non-null  int64  
 1   title                 1118002 non-null  object 
 2   vote_average          1118015 non-null  float64
 3   vote_count            1118015 non-null  int64  
 4   status                1118015 non-null  object 
 5   release_date          939457 non-null   object 
 6   revenue               1118015 non-null  int64  
 7   runtime               1118015 non-null  int64  
 8   adult                 1118015 non-null  bool   
 9   backdrop_path         298523 non-null   object 
 10  budget                1118015 non-null  int64  
 11  homepage              118817 non-null   object 
 12  imdb_id               597826 non-null   object 
 13  original_language     1118015 non-null  object 
 14  original_title        1118002 non-

In [106]:
df_tmdb.columns


Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [11]:
df_tmdb['production_countries'].unique()

array(['United Kingdom, United States of America',
       'United States of America, United Kingdom',
       'United States of America', ...,
       'Greece, Mexico, United States of America', 'Switzerland, Tunisia',
       'Bulgaria, Germany, United States of America'], dtype=object)

In [12]:
#define df_tmdb_latam
df_tmdb_latam = pd.DataFrame()
#columns we want to keep
columns_to_keep =['imdb_id', 'popularity', 'poster_path', 'production_countries']
df_tmdb = df_tmdb.loc[:,columns_to_keep]
#on ne garde que les pays  dans country_code_list
# et on identifie les pays pour lesquels on n'a aucun film

#drop na in df_tmdb['production_countries'] #remove rows with missing production country
df_tmdb.dropna(subset=['production_countries'], inplace=True)       
for country_name in  country_names:
  mask = df_tmdb['production_countries'].str.contains(country_name)
  #concat
  if (df_tmdb_latam.empty):
    df_tmdb_latam = df_tmdb[mask].copy()
  else:
    df_tmdb_latam = pd.concat([df_tmdb_latam, df_tmdb[mask]])
df_tmdb_latam.reset_index(drop=True, inplace= True)
df_tmdb_latam.info()


df_tmdb_latam.head(3)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31303 entries, 0 to 31302
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               20145 non-null  object 
 1   popularity            31303 non-null  float64
 2   poster_path           24969 non-null  object 
 3   production_countries  31303 non-null  object 
dtypes: float64(1), object(3)
memory usage: 978.3+ KB


,imdb_id,popularity,poster_path,production_countries
0,tt3011894,15.063,/pvSty5MCAxdj8LlAEXCDfM1YNOl.jpg,"Argentina, Spain"
1,tt1305806,17.242,/dkeAwfZzwL3WvToydE3CXiY80E0.jpg,"Argentina, Spain"
2,tt0168629,18.067,/8Wdd3fQfbbQeoSfWpHrDfaFNhBU.jpg,"Argentina, Denmark, Finland, France, Germany, ..."


In [13]:
df_tmdb_latam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31303 entries, 0 to 31302
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               20145 non-null  object 
 1   popularity            31303 non-null  float64
 2   poster_path           24969 non-null  object 
 3   production_countries  31303 non-null  object 
dtypes: float64(1), object(3)
memory usage: 978.3+ KB


In [14]:
df_tmdb_latam['production_countries'].unique()

array(['Argentina, Spain',
       'Argentina, Denmark, Finland, France, Germany, Iceland, Italy, Netherlands, Norway, Sweden, United Kingdom, United States of America',
       'Argentina, Chile, France, Germany, Peru, United Kingdom, United States of America',
       ..., 'Canada, France, Venezuela', 'France, Lebanon, Venezuela',
       'Portugal, Spain, Venezuela'], dtype=object)

In [16]:

#Merge
#inner join pour garder uniquement les ratings pour les films qui sont dans le df

df_merged_movies= pd.merge(df_title_basics_movies,df_tmdb_latam, how= 'inner',left_on='tconst',right_on= 'imdb_id')
#drop imdb_id
df_merged_movies.drop(columns=['imdb_id'],inplace=True)
df_merged_movies.reset_index(drop=True, inplace=True)
# del(df_tmdb_latam)
# del(df_title_basics_movies)
#format year as integer
df_merged_movies.dropna(subset=['startYear'],inplace=True)
df_merged_movies['startYear'] = df_merged_movies['startYear'].astype(int)

display(df_merged_movies.head(3))
display(df_merged_movies.info())

# df_merged_movies.drop(columns= ['titleId','region'],inplace=True)
# df_merged_movies.info()

# #garder le titre en FR (title) s'il n'est pas null, sinon primary title
# df_merged_movies['title'] = df_merged_movies['title'].fillna(df_merged_movies['primaryTitle'])
# df_merged_movies.info()


,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico
1,tt0007638,movie,El amor que huye,1917,Drama,1.400,NaN,Mexico
2,tt0007646,movie,El apóstol,1917,"Animation,Comedy,Drama",0.840,/xOyr2LTVbHiAiHY8mLNGqt68Llg.jpg,Argentina


<class 'pandas.core.frame.DataFrame'>
Index: 14933 entries, 0 to 15226
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tconst                14933 non-null  object 
 1   titleType             14933 non-null  object 
 2   primaryTitle          14933 non-null  object 
 3   startYear             14933 non-null  int64  
 4   genres                14933 non-null  object 
 5   popularity            14933 non-null  float64
 6   poster_path           12810 non-null  object 
 7   production_countries  14933 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 1.0+ MB


None

## Title akas - Maybe not needed if we keep original title only, but brazilian movies !
(traduction des titres en Espagnol ? plutot qu'en français? ou les deux)
Il ya très peu de titres traduits en anglais 3697/14993

In [19]:
url = 'https://datasets.imdbws.com/title.akas.tsv.gz'

chunk_size = 10000
iterator = pd.read_csv(url, sep='\t', compression='gzip', iterator=True, chunksize=chunk_size, na_values=["\\N"])

cleaned_chunks = []
for chunk in iterator:

    if 'attributes' in chunk.columns:
        chunk.drop(columns=['attributes'], inplace=True)      #drop attributes

    if 'types' in chunk.columns:
        #Peggi
        chunk.drop(chunk[chunk['types']!='imdbDisplay'].index, inplace=True)      #drop rows with types different from imdbDisplay
         ##
        chunk.drop(columns=['types'], inplace=True)      #drop types

    if 'isOriginalTitle' in chunk.columns:
        chunk.drop(chunk[chunk['isOriginalTitle'] == 1].index, inplace=True)      #drop rows with isOriginalTitle

    if 'region' in chunk.columns:
        #keep all raws whose region is in country_code_list
        region_list = ['US' # we keep only the title used in the US
                    #     'AR','AW','BZ','BO','BR','CL','CO',
                    #   'CR','CU','DO','EC','SV','FK','GF',
                    #   'GT','GY','HT','HN','MX','AN','NI',
                    #   'PA','PY','PE','GS','SR','TT','UY','VE'
                      ]
        chunk = chunk.loc[chunk['region'].isin(region_list)]

    cleaned_chunks.append(chunk)

#on concatene
full_clean_title_akas = pd.concat(cleaned_chunks, ignore_index=True)

In [20]:
full_clean_title_akas['region'] = full_clean_title_akas['region'].astype(str)
full_clean_title_akas = full_clean_title_akas.drop(columns=['ordering','language','isOriginalTitle'])
full_clean_title_akas.head()

,titleId,title,region
0,tt0000001,Carmencita,US
1,tt0000003,Poor Pierrot,US
2,tt0000005,Blacksmith Scene,US
3,tt0000006,Chinese Opium Den,US
4,tt0000007,Corbett and Courtney Before the Kinetograph,US


In [21]:
#Merge
#inner join pour garder tous les films qui sont dans le df
df_merged_movies_2= pd.merge(df_merged_movies,full_clean_title_akas, how= 'left',left_on='tconst',right_on= 'titleId')
#drop title_id
df_merged_movies_2.drop(columns=['titleId'],inplace=True)
df_merged_movies_2.reset_index(drop=True, inplace=True)
del(full_clean_title_akas)

display(df_merged_movies_2.head(3))
display(df_merged_movies_2.info())

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,title,region
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN
1,tt0007638,movie,El amor que huye,1917,Drama,1.400,NaN,Mexico,NaN,NaN
2,tt0007646,movie,El apóstol,1917,"Animation,Comedy,Drama",0.840,/xOyr2LTVbHiAiHY8mLNGqt68Llg.jpg,Argentina,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14994 entries, 0 to 14993
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tconst                14994 non-null  object 
 1   titleType             14994 non-null  object 
 2   primaryTitle          14994 non-null  object 
 3   startYear             14994 non-null  int64  
 4   genres                14994 non-null  object 
 5   popularity            14994 non-null  float64
 6   poster_path           12869 non-null  object 
 7   production_countries  14994 non-null  object 
 8   title                 3697 non-null   object 
 9   region                3697 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.1+ MB


None

In [22]:
df_merged_movies_2.isna().sum()

tconst                      0
titleType                   0
primaryTitle                0
startYear                   0
genres                      0
popularity                  0
poster_path              2125
production_countries        0
title                   11297
region                  11297
dtype: int64

## On récupère title principals 
(actors and actresses, writer, director)

In [32]:
url = 'https://datasets.imdbws.com/title.principals.tsv.gz'
chunk_size = 10000
iterator = pd.read_csv(url, sep='\t', compression='gzip', iterator=True, chunksize=chunk_size, na_values=["\\N"])

In [33]:
cleaned_chunks = []
for chunk in iterator:
    # numeric_cols = chunk.select_dtypes(include=['number']).columns    #selectionner les colonne numerique
    if 'job' in chunk.columns:
        chunk.drop(columns=['job'], inplace=True)      #drop job

    #on supprime la colonne knownForTitles
    chunk = chunk.loc[(chunk['category'] =='actor') | (chunk['category']== 'actress') | (chunk['category']== 'director')].copy()
    chunk.drop(columns=['ordering','characters'], inplace = True) #we keep 'category'    
    #############
    cleaned_chunks.append(chunk)
    del(chunk)

In [34]:

subset_actors_directors = pd.concat(cleaned_chunks, ignore_index=True)      #concaterner mes chunks pour avoir un full dataset actors clean
print(subset_actors_directors.duplicated().value_counts())
#On supprime les doublons
subset_actors_directors= subset_actors_directors.drop_duplicates(subset=None, keep="first", inplace=False)
subset_actors_directors.reset_index(drop=True, inplace=True)

subset_actors_directors.info(verbose=True, show_counts=True)

False    43269077
True      1450537
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43269077 entries, 0 to 43269076
Data columns (total 3 columns):
 #   Column    Non-Null Count     Dtype 
---  ------    --------------     ----- 
 0   tconst    43269077 non-null  object
 1   nconst    43269077 non-null  object
 2   category  43269077 non-null  object
dtypes: object(3)
memory usage: 990.4+ MB


In [35]:
subset_actors_directors.head()

,tconst,nconst,category
0,tt0000001,nm0005690,director
1,tt0000002,nm0721526,director
2,tt0000003,nm0721526,director
3,tt0000004,nm0721526,director
4,tt0000005,nm0443482,actor


In [36]:
# inner join? it seems enough as we only want movies with actors ;p
df_merged_movies_3 = pd.merge(df_merged_movies_2,subset_actors_directors, how= 'inner',on='tconst')

# del(subset_actors_directors)

df_merged_movies_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129618 entries, 0 to 129617
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   tconst                129618 non-null  object 
 1   titleType             129618 non-null  object 
 2   primaryTitle          129618 non-null  object 
 3   startYear             129618 non-null  int64  
 4   genres                129618 non-null  object 
 5   popularity            129618 non-null  float64
 6   poster_path           112913 non-null  object 
 7   production_countries  129618 non-null  object 
 8   title                 32949 non-null   object 
 9   region                32949 non-null   object 
 10  nconst                129618 non-null  object 
 11  category              129618 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 11.9+ MB


In [37]:

df_merged_movies_3.head()

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,title,region,nconst,category
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0220230,actress
1,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0846016,actor
2,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0704415,actor
3,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0749618,actress
4,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0035999,actor


## Name basics

In [38]:
url = 'https://datasets.imdbws.com/name.basics.tsv.gz'
df_name_basicschunk_size = 1000
iterator = pd.read_csv(url, sep='\t', compression='gzip', iterator=True, chunksize=chunk_size, na_values=["\\N"])

In [39]:
cleaned_chunks = []

for chunk in iterator:
    if 'primaryName' in chunk.columns:
        chunk.dropna(subset=['primaryName'], inplace=True)

    # if 'primaryProfession' in chunk.columns:
        # chunk.dropna(subset=['primaryProfession'], inplace=True)
    chunk.drop(['knownForTitles','deathYear','birthYear','primaryProfession'], axis=1, inplace=True)
    # chunk['birthYear'] = pd.to_datetime(chunk['birthYear'], format='%Y', errors='coerce')

    cleaned_chunks.append(chunk)

In [40]:
df_name_basics = pd.concat(cleaned_chunks, ignore_index=True)
display(df_name_basics.head(10))
print(df_name_basics.info())
print(df_name_basics.describe())

,nconst,primaryName
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall
2,nm0000003,Brigitte Bardot
3,nm0000004,John Belushi
4,nm0000005,Ingmar Bergman
5,nm0000006,Ingrid Bergman
6,nm0000007,Humphrey Bogart
7,nm0000008,Marlon Brando
8,nm0000009,Richard Burton
9,nm0000010,James Cagney


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13886829 entries, 0 to 13886828
Data columns (total 2 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   nconst       object
 1   primaryName  object
dtypes: object(2)
memory usage: 211.9+ MB
None
           nconst primaryName
count    13886829    13886829
unique   13886829    10669548
top     nm9993719        Alex
freq            1         523


In [41]:

# jointure  name_basics avec df_merged_series
df_merged_movies_4 = pd.merge(df_merged_movies_3,df_name_basics, how= 'inner',on='nconst') #left_on='a_id',right_on='nconst')

df_merged_movies_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129618 entries, 0 to 129617
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   tconst                129618 non-null  object 
 1   titleType             129618 non-null  object 
 2   primaryTitle          129618 non-null  object 
 3   startYear             129618 non-null  int64  
 4   genres                129618 non-null  object 
 5   popularity            129618 non-null  float64
 6   poster_path           112913 non-null  object 
 7   production_countries  129618 non-null  object 
 8   title                 32949 non-null   object 
 9   region                32949 non-null   object 
 10  nconst                129618 non-null  object 
 11  category              129618 non-null  object 
 12  primaryName           129618 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB


In [42]:
df_merged_movies_4.head()

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,title,region,nconst,category,primaryName
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0220230,actress,Mimí Derba
1,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0846016,actor,Julio Taboada
2,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0704415,actor,Salvador Quiroz
3,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0749618,actress,Emilia Ruiz del Castillo
4,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0035999,actor,Salvador Arnaldo


In [43]:
df_merged_movies_4 = df_merged_movies_4.drop_duplicates(ignore_index=True)
df_merged_movies_4.head()

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,title,region,nconst,category,primaryName
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0220230,actress,Mimí Derba
1,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0846016,actor,Julio Taboada
2,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0704415,actor,Salvador Quiroz
3,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0749618,actress,Emilia Ruiz del Castillo
4,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0035999,actor,Salvador Arnaldo


In [44]:
df_merged_movies_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121857 entries, 0 to 121856
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   tconst                121857 non-null  object 
 1   titleType             121857 non-null  object 
 2   primaryTitle          121857 non-null  object 
 3   startYear             121857 non-null  int64  
 4   genres                121857 non-null  object 
 5   popularity            121857 non-null  float64
 6   poster_path           105505 non-null  object 
 7   production_countries  121857 non-null  object 
 8   title                 29511 non-null   object 
 9   region                29511 non-null   object 
 10  nconst                121857 non-null  object 
 11  category              121857 non-null  object 
 12  primaryName           121857 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 12.1+ MB


In [53]:
#regrouper par nom de film créér une colonne avec category=='actors' or 'actress'
columns_to_keep = ['tconst','title',
                    'primaryTitle','startYear','genres',
                    'production_countries','popularity','poster_path',
                    ]
# Regroupez par les colonnes spécifiées et concaténez les noms des acteurs et directeurs
mask_both_actors = (df_merged_movies_4['category'] == 'actor' ) | (df_merged_movies_4['category'] == 'actress')
df_grouped = df_merged_movies_4.groupby(columns_to_keep, as_index=False).agg(
    actors=('primaryName', lambda x: ', '.join(x[mask_both_actors])),
    directors=('primaryName', lambda x: ', '.join(x[df_merged_movies_4['category'] == 'director']))
)

# del(df_merged_movies)
df_grouped.head()

,tconst,title,primaryTitle,startYear,genres,production_countries,popularity,poster_path,actors,directors
0,tt0013417,My American Wife,My American Wife,1922,Drama,"Argentina, United States of America",0.866,/bVECaNPfOJwbDyciolVFAGlG7UG.jpg,"Gloria Swanson, Antonio Moreno, Josef Swickard...",Sam Wood
1,tt0018503,The Ghost Train,The Ghost Train,1927,Romance,Mexico,0.600,/lC7VHgXqqOmwowPGUDhJlAlZlMt.jpg,"Carlos Sánchez Alducín, Rafael Araiza, Carrera...",Gabriel García Moreno
2,tt0022756,¡Que viva Mexico!,¡Que viva Mexico!,1932,History,"Mexico, United States of America",1.784,/dNfI9Jiv8TyYuGi3wFqVAgiIshu.jpg,"Félix Balderas, Sara García, Martín Hernández,...","Grigoriy Aleksandrov, Sergei Eisenstein"
3,tt0023902,Godfather Mendoza,Godfather Mendoza,1934,"Drama,War",Mexico,1.382,/b56SuoyQWualtXkHmrVN6HROZiD.jpg,"Alfredo del Diestro, Carmen Guerrero, Antonio ...","Juan Bustillo Oro, Fernando de Fuentes"
4,tt0025062,Two Monks,Two Monks,1934,"Drama,Horror,Mystery",Mexico,2.017,/mv1gSN5Bz3Btoacx7T5flr9PWcg.jpg,"Víctor Urruchúa, Carlos Villatoro, Magda Halle...",Juan Bustillo Oro


In [54]:
print(df_grouped.duplicated().value_counts())

False    3146
Name: count, dtype: int64


In [55]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3146 entries, 0 to 3145
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tconst                3146 non-null   object 
 1   title                 3146 non-null   object 
 2   primaryTitle          3146 non-null   object 
 3   startYear             3146 non-null   int64  
 4   genres                3146 non-null   object 
 5   production_countries  3146 non-null   object 
 6   popularity            3146 non-null   float64
 7   poster_path           3146 non-null   object 
 8   actors                3146 non-null   object 
 9   directors             3146 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 245.9+ KB


## Title ratings

In [56]:
url_ratings = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
df_title_ratings = pd.read_csv(url_ratings, sep='\t', compression='gzip', na_values=["\\N"])

#inner join pour garder uniquement les ratings pour les films qui sont dans le df
df_merged_popular = pd.merge(df_grouped,df_title_ratings, how= 'inner',on='tconst')

# del(df_title_ratings)
# del(df_grouped)
display(df_merged_popular.head(3))
display(df_merged_popular.info())

,tconst,title,primaryTitle,startYear,genres,production_countries,popularity,poster_path,actors,directors,averageRating,numVotes
0,tt0013417,My American Wife,My American Wife,1922,Drama,"Argentina, United States of America",0.866,/bVECaNPfOJwbDyciolVFAGlG7UG.jpg,"Gloria Swanson, Antonio Moreno, Josef Swickard...",Sam Wood,4.8,20
1,tt0018503,The Ghost Train,The Ghost Train,1927,Romance,Mexico,0.600,/lC7VHgXqqOmwowPGUDhJlAlZlMt.jpg,"Carlos Sánchez Alducín, Rafael Araiza, Carrera...",Gabriel García Moreno,6.4,61
2,tt0023902,Godfather Mendoza,Godfather Mendoza,1934,"Drama,War",Mexico,1.382,/b56SuoyQWualtXkHmrVN6HROZiD.jpg,"Alfredo del Diestro, Carmen Guerrero, Antonio ...","Juan Bustillo Oro, Fernando de Fuentes",7.6,622


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3033 entries, 0 to 3032
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tconst                3033 non-null   object 
 1   title                 3033 non-null   object 
 2   primaryTitle          3033 non-null   object 
 3   startYear             3033 non-null   int64  
 4   genres                3033 non-null   object 
 5   production_countries  3033 non-null   object 
 6   popularity            3033 non-null   float64
 7   poster_path           3033 non-null   object 
 8   actors                3033 non-null   object 
 9   directors             3033 non-null   object 
 10  averageRating         3033 non-null   float64
 11  numVotes              3033 non-null   int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 284.5+ KB


None

## Ecriture du df trié par popularité dans un fichier csv 

In [57]:
df_merged_popular = df_merged_popular.sort_values(by=['averageRating','numVotes'], ascending=False)

df_merged_popular.to_csv("grouped_movies_latam_ML.csv",index=False,sep =';')